In [1]:
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.grid_objs import Grid, Column
import time

# Generate random walk with numpy

In [2]:
np.random.seed(0)
brownian_motion = np.cumsum(np.random.uniform(low=-0.5, high=0.5, size=(100,3)), axis=0)

# Build the plotly grid

In [3]:
my_columns = []
nr_frames = brownian_motion.shape[0]
for k in range(nr_frames):
    my_columns.extend(
        [Column(brownian_motion[:k,0], 'x{}'.format(k+1)),
         Column(brownian_motion[:k,1], 'y{}'.format(k+1)),
         Column(brownian_motion[:k,2], 'z{}'.format(k+1))
        ])
grid = Grid(my_columns)
py.grid_ops.upload(grid, 'anim_random'+str(time.time()), auto_open=False)

'https://plot.ly/~dadelforge/84/'

# Make the figure

In [20]:
data = [go.Scatter3d(
    xsrc = grid.get_column_reference('x1'),
    ysrc = grid.get_column_reference('y1'),
    zsrc = grid.get_column_reference('z1'),
    marker=dict(
        size=4,
    ),
    line=dict(
        color='#1f77b4',
        width=1
    )
)]


frames=[
    {
        'data':[
            go.Scatter3d(
                xsrc = grid.get_column_reference('x{}'.format(k + 1)),
                ysrc = grid.get_column_reference('y{}'.format(k + 1)),
                zsrc = grid.get_column_reference('z{}'.format(k + 1))
            )],
        'name':'frame{}'.format(k + 1)
    } for k in range(nr_frames)]

sliders=[
    dict(
        steps=[dict(method='animate',
                    args= [['frame{}'.format(k + 1)],
                            dict(mode='immediate',
                                 frame= dict(duration=nr_frames, redraw= False),
                                 transition=dict(duration=0))],
                    label='{:d}'.format(k+1)) for k in range(nr_frames)], 
        transition= dict(duration=0),
        x=0,
        y=0, 
        currentvalue=dict(font=dict(size=12), 
                          prefix='Time: ', 
                          visible=True, 
                          xanchor='center'
                         ),  
        len=1.0
    )
]

axis3d = dict(
    showbackground= True,
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",      
    zerolinecolor="rgb(255, 255, 255)",  
)

layout3d = dict(
         title='Random walk', 
         width=600,
         height=600,
         scene=dict(xaxis=dict(axis3d, **dict(range=[min(brownian_motion[:,0]), max(brownian_motion[:,0])], autorange=False)), 
                    yaxis=dict(axis3d, **dict(range=[min(brownian_motion[:,1]), max(brownian_motion[:,1])], autorange=False)), 
                    zaxis=dict(axis3d, **dict(range=[min(brownian_motion[:,2]), max(brownian_motion[:,2])], autorange=False)), 
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus=[
             dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=1.3,
                  xanchor='right',
                  yanchor='top',
                  pad=dict(t=0, r=10),
                  buttons=[dict(label='Play',
                                method='animate',
                                args=[
                                    None, 
                                    dict(frame=dict(duration=nr_frames, redraw=False),
                                         transition=dict(duration=0),
                                         fromcurrent=True,
                                         mode='immediate')
                                ])])
         ],
        sliders=sliders
)


fig=dict(data=data, layout=layout3d, frames=frames)
py.icreate_animations(fig, filename='anim_trajectory3d'+str(time.time()))